## IBM Data Science Capstone Project


## Best Nieghborhood for technology Company in Toronto

**Lemlem Ayele**.  
December 8th, 2019

## 1.1 Introduction
With over $8 billion in contributions to annual employee wages, Toronto's technology sector is one of the fast-growing in north America. Its great infrastructure, the ease of connection to the rest of the world, good transit, and strong business-centric nature makes Toronto very attractive to international firms and companies. DexTR, a technology company based in San Francisco: engaged in design graphics processing units (GPUs), system chip for smartphone and vehicle navigation processors, is looking to open the second largest center in North America.

After months of regress research and analysis, DexTR has been decided Toronto as the next logical location for expansion in North America. Now that the decision has been made, the objective of this capstone is to analyze and select the optimal location for the DexTR office in the city of Toronto. While not limited to, the criteria below are some of the guiding measures.



### 1.2 Problem.

After months of regress research and analysis, DexTR has been decided Toronto as the next logical location for expansion in North America. Now that the decision has been made, the objective of this capstone is to analyze and select the optimal location for the DexTR office in the city of Toronto. While not limited to, the criteria below are some of the guiding measures.

----    
* Neighborhood score
* Accessibility and public transit
* Venues
* Crime


### 1.3 Interest

<p>
The result of this analysis is of great interest to DexTR, using the above-listed criteria, the analysis will provide some useful measurable data which will aid the company in making an informed decision on the next office location.
While DexTR has a vested interest at this point, this analysis will also provide helpful information for any startup or business considering Toronto. It can also have some useful insight for real estate companies, existing businesses, communities, and individuals.
</p>

## 2.0 Data Acquisition and wrangling

## Data Source
The data for the city of Toronto neighborhood was obtained by scraping the Wikipedia page for the postal code of the city of Toronto, which consisted of corresponding borough and neighborhood. Additional dataset to evaluate crime and population density is obtained from the Toronto Police service: public safety data portoal. While the crime dataset contains over hundred fifty thousand features with dataset between 2014 -2018, only two features were used in our analysis as those are relevant to our analysis. 

#### Public safty data portal: <a href="http://data.torontopolice.on.ca/datasets/98f7dde610b54b9081dfca80be453ac9_0" target="blank">data portoal.</a>
#### Foursquare API: <a href="https://api.foursquare.com/v2/venues/explore" target="blank">Foursquare.</a>
#### Walk score: <a href="http://api.walkscore.com" target="blank">Walk Score</a>
#### Toronto Nieghborhoods data: <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" target="blank">Wikipedia</a>

#### 2.2 Data Wrangling
<p>
The CSV file obtained by scraping the Wikipedia page was transformed into a pandas dataframe. They're some missing data, for example, rows with no assigned borough and neighborhood were dropped. While rows with postal code and borough, but no neighborhood, the neighborhood was set the same as its borough.
As for the crime dataset, the crime for each community was obtained, and cleaned by dropping any row that was not assigned.
</p>
We will then use a foursquare API to explore the different neighborhoods and boroughs to gain insight. Then we will do farther analysis by merging the two datasets for more in-depth insight.


In [76]:
# The code was removed by Watson Studio for sharing.

In [10]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #wikipedia link to the table
headers = {'User-Agent': 'Mozilla/5.0'} #setting my browser

In [11]:
response = requests.get(url, headers = headers)
soup = BeautifulSoup(response.content, "html.parser")

In [12]:
# creating empty list for the three in the table to store data
postCodeList = []
boroughList = []
neighborList = []

In [13]:
## accessing the table and saving it under vairable
wikitable = soup.find('table', class_="wikitable")

In [14]:
# iterating through the table to get the contents
for row in wikitable.find_all("tr"): #finding row in the table
    cell = row.find_all("td") #finding cells in the row
    if(len(cell) > 0):
        #appending list to corresponding column
        postCodeList.append(cell[0].text)
        boroughList.append(cell[1].text)
        neighborList.append(cell[2].text.rstrip('\n'))

### Transforming the data in to pandas DataFram

In [80]:
#creating columns for corrosponding lists
columns ={"PostCode": postCodeList, "Borough": boroughList, "Neighborhood": neighborList}
wiki_df = pd.DataFrame(columns)
wiki_df.head(10)

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Downtown Toronto,Queen's Park



## Cleaning data
If you have noticed, the first two rows on the above table both have borough and neighborhood 'not assigned', as discussed above, both those rows were dropped. The second and third row have now moved up to first and second position as you can see below with the cleaned datafram sample. 
respectively, for a borough with no neighborhood assigned, we set borough as neighborhood. 

In [83]:
# drop cells with borough not assigned
wiki_df_dropna = wiki_df[wiki_df.Borough != "Not assigned"].reset_index(drop=True)
wiki_df_dropna.head(10)

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [77]:
# group neighborhoods with same postal code area
wiki_df_grouped = wiki_df_dropna.groupby(["PostCode", "Borough"],sort=False, as_index=False).agg(lambda x: ", ".join(x))
wiki_df_grouped.reset_index(inplace=True)
#wiki_df_grouped.head(10)

In [82]:
# for a borough with no  neighborhood assigned, set borough as neighborhood 
for index, row in wiki_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
wiki_df_grouped.head(10).set_index("PostCode", inplace=True)
#wiki_df_grouped.head(30)

## 3.0 Analysis
We also grouped neighborhoods into its corresponding borough to gain more insight. As you can see below, the following table makes it much easier to understand the number of neighborhoods in each borough.


In [21]:
wiki_df_dropna.groupby(["Borough"]).count()


,PostCode,Neighborhood
Borough,,
Central Toronto,17,17
Downtown Toronto,37,37
East Toronto,7,7
East York,6,6
Etobicoke,44,44
Mississauga,1,1
North York,38,38
Queen's Park,1,1
Scarborough,37,37


In [25]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# Load the coordinates from the csv file
coordinates = pd.read_csv(Geospatial_Coordinates)

# renaming postal code column
coordinates.rename(columns={"Postal Code": "PostCode"}, inplace=True)
#coordinates.head()


## Merging the table and coordinates for the city of Toronto
Based on our criteria described in the problem section, the Downtown Toronto borough was selected for farther exploration. This borough consists of 37 neighborhoods (please see the table above.).
We are going to merge neighborhood data with geo coordinates of corrosponding location.

In [27]:
#Merging the table and coordinates
merged_df = wiki_df_grouped.merge(coordinates, on="PostCode", how="left")
merged_df.head(20)

,index,PostCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,4,M7A,Queen's Park,Not assigned,43.662301,-79.389494
5,5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
6,6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,7,M3B,North York,Don Mills North,43.745906,-79.352188
8,8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Map of Toronto with neighborhoods
Here is an interactive map for the city of Toronto that you can explore.




In [29]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='#6a737d',
        fill=True,
        fill_color='#007849',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Exploring the downtown and clustering the neighborhoods using geopy.
Now that we have selected what borough we're going to focus on, we are going to define geolocation for our borough of downtown Toronto. We will use the Foursquare API to explore the neighborhoods. 
And Segment and cluster neighborhoods in downtown Toronto

In [28]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate for city of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate for city of Toronto are 43.653963, -79.387207.


In [85]:
## Using Foursquare API to explore downtown Toronto neighborhood

In [30]:
# The code was removed by Watson Studio for sharing.

In [86]:
## Segment and cluster neighborhoods in downtown Toronto

In [31]:
Downtown_Toronto = merged_df[merged_df["Borough"] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto

,index,PostCode,Borough,Neighborhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
2,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,36,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
9,42,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576


In [87]:
## Geographical coordinates of downtown Toronto

In [33]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of downtown Toronto are 43.6563221, -79.3809161.


### Creating map for downtown using the corrosponding latitude and longitude values
Using the above geographical coordinate of downtown Toronto (43.6563221, -79.3809161). We have created an interactive map for our target borough. You can zoom in & out, and interact with each popup.

In [34]:
# create map of downtown Toronto with latitude & longitude values
map_Downtown_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Downtown_Toronto['Latitude'], Downtown_Toronto['Longitude'], Downtown_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_Toronto)  
    
map_Downtown_Toronto

In [89]:
#Downtown_Toronto.loc[7, 'Neighborhood']

In [91]:
# # get geo coordinate for Willowdale south
# neighborhood_latitude = Downtown_Toronto.loc[7, 'Latitude'] # neighborhood latitude value
# neighborhood_longitude = Downtown_Toronto.loc[7, 'Longitude'] # neighborhood longitude value

# neighborhood_name = Downtown_Toronto.loc[7, 'Neighborhood'] # neighborhood name

# print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
#                                                                neighborhood_latitude, 
#                                                                neighborhood_longitude))

###  Explore top 500 venues in downtown Toronto with in 2km radius.
By utilizing the foursquare API, its time to explore the venues in downtown Toronto. Due to the large data, we limit the venues to the top 500 within a 2km radius. We then extract the venue category.
In the table below, we will display ten samples of the venue categories that we have extracted using the foursquare API. It also contains the name along with geolocation.


In [37]:
# Explore top 100 venues in downtown Toronto with in 999 meters
radius = 1999
LIMIT = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [70]:
results = requests.get(url).json()


In [39]:
# function to get venue category 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
1,Downtown Toronto,Neighborhood,43.653232,-79.385296
2,Nathan Phillips Square,Plaza,43.652270,-79.383516
3,Shangri-La Toronto,Hotel,43.649129,-79.386557
4,Soho House Toronto,Speakeasy,43.648734,-79.386541
5,Rosalinda,Vegetarian / Vegan Restaurant,43.650252,-79.385156
6,The Keg Steakhouse & Bar,Steakhouse,43.649937,-79.384196
7,Friendly Stranger - Cannabis Culture Shop,Smoke Shop,43.650387,-79.388523
8,Pizzeria Libretto,Pizza Place,43.648334,-79.385111
9,JaBistro,Sushi Restaurant,43.649687,-79.388090


In [92]:
#print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

## Neighborhoods and Venues
Here we can explore the downtown neighborhood and group the surrounding venue.


In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
Downtown_Toronto_venues = getNearbyVenues(names=Downtown_Toronto['Neighborhood'],
                                   latitudes=Downtown_Toronto['Latitude'],
                                   longitudes=Downtown_Toronto['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


In [95]:
## Grouping venues for each nighborhood

In [45]:
Downtown_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94
Christie,17,17,17,17,17,17
Church and Wellesley,84,84,84,84,84,84
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [96]:
## Analyzing Each Neighborhood

In [ ]:
# one hot encoding
Downtown_Toronto_onehot = pd.get_dummies(Downtown_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_Toronto_onehot['Neighborhood'] = Downtown_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_Toronto_onehot.columns[-1]] + list(Downtown_Toronto_onehot.columns[:-1])
Downtown_Toronto_onehot = Downtown_Toronto_onehot[fixed_columns]

#Downtown_Toronto_onehot.head()

In [71]:
Downtown_grouped = Downtown_Toronto_onehot.groupby('Neighborhood').mean().reset_index()


## Cluster  and anlysing neighborhoods


In [ ]:
# set number of clusters
kclusters = 5

Downtown_grouped_clustering = Downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_grouped['Neighborhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Salad Place,Bakery,Restaurant,Asian Restaurant,Sushi Restaurant
1,Berczy Park,Coffee Shop,Steakhouse,Cheese Shop,Farmers Market,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Café,French Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
3,"Cabbagetown, St. James Town",Coffee Shop,Café,Italian Restaurant,Chinese Restaurant,Bakery,Restaurant,Pub,Pizza Place,Outdoor Sculpture,Sandwich Place
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Ice Cream Shop,Bubble Tea Shop,Gym / Fitness Center,Salad Place,Japanese Restaurant
5,"Chinatown, Grange Park, Kensington Market",Café,Bar,Dumpling Restaurant,Bakery,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Record Shop
6,Christie,Grocery Store,Café,Park,Coffee Shop,Restaurant,Diner,Baby Store,Convenience Store,Italian Restaurant,Nightclub
7,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Gastropub,Men's Store,Bubble Tea Shop,Mediterranean Restaurant,Gym
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Italian Restaurant,Seafood Restaurant,Steakhouse,Gym
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Bar,Seafood Restaurant,Gastropub,Deli / Bodega,Bakery,American Restaurant


In [97]:
## Create new dataframe cluster each neighborhood

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Downtown_merged = Downtown_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Downtown_merged = Downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Downtown_merged.head() # check the last columns!

## Visualize the clusters results

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(Downtown_merged['Latitude'], Downtown_merged['Longitude'], Downtown_merged['PostCode'], Downtown_merged['Borough'], Downtown_merged['Neighborhood'], Downtown_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters
## Clusters 1
 Although there were five clusters used in this model, we will only display one cluster (found at the index of 0)in this report for the sake of efficiency. As this provides sufficient sample information.

In [54]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 0, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,PostCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Chocolate Shop
2,M5B,-79.378937,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Pizza Place,Bubble Tea Shop,Italian Restaurant,Tea Room
3,M5C,-79.375418,0.0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,American Restaurant,Beer Bar,Diner,Italian Restaurant,Cosmetics Shop
4,M5E,-79.373306,0.0,Coffee Shop,Steakhouse,Cheese Shop,Farmers Market,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Café,French Restaurant
5,M5G,-79.387383,0.0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Ice Cream Shop,Bubble Tea Shop,Gym / Fitness Center,Salad Place,Japanese Restaurant
7,M5H,-79.384568,0.0,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Salad Place,Bakery,Restaurant,Asian Restaurant,Sushi Restaurant
8,M5J,-79.381752,0.0,Coffee Shop,Aquarium,Hotel,Café,Brewery,Restaurant,Italian Restaurant,Scenic Lookout,Fried Chicken Joint,Park
9,M5K,-79.381576,0.0,Coffee Shop,Café,Hotel,Restaurant,Bar,Seafood Restaurant,Gastropub,Deli / Bodega,Bakery,American Restaurant
10,M5L,-79.379817,0.0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Italian Restaurant,Seafood Restaurant,Steakhouse,Gym
12,M5T,-79.400049,0.0,Café,Bar,Dumpling Restaurant,Bakery,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Record Shop


In [98]:
## Clusters 2

In [ ]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 1, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

In [99]:
## Clusters 3

In [ ]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 2, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

In [ ]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 3, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

In [ ]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 4, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

## Crime data from Public safty data portal 
The crime dataframe consists of 167525 features between 2014 - 2018, I'm going to limit this to just 500 incidents due to computational cost. For the purpose of this analysis, we will focus on what's relevant to us. We will use using the feature MCI (major crime incident), which includes everything... I.e theft, robbery, assault. We will then cluster this for visualization.


In [59]:

body = client_e2121f57bafa41a3b4483b72a3a1a8a6.get_object(Bucket='courseracapstone-donotdelete-pr-xz5rrczk33f99i',Key='MCI_2014_to_2018.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_crime = pd.read_csv(body)
df_crime.head()


,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,reportedhour,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Lat,Long,ObjectId
0,-79.385193,43.659229,2349,GO-20149004286,2014-06-20T10:55:00.000Z,2014-06-20T13:20:00.000Z,Apartment,2130,210,Theft Over,2014,June,20,171,Friday,13,2014.0,June,20.0,171.0,Friday,10,Theft Over,D52,76,Bay Street Corridor (76),43.659229,-79.385193,2001
1,-79.425400,43.777592,2350,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1457,100,Pointing A Firearm,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Assault,D32,36,Newtonbrook West (36),43.777592,-79.425400,2002
2,-79.425400,43.777592,2351,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1610,100,Robbery With Weapon,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Robbery,D32,36,Newtonbrook West (36),43.777592,-79.425400,2003
3,-79.210373,43.801727,2352,GO-20142412127,2014-07-02T01:30:00.000Z,2014-07-02T05:40:00.000Z,House,2120,200,B&E,2014,July,2,183,Wednesday,5,2014.0,July,2.0,183.0,Wednesday,1,Break and Enter,D42,132,Malvern (132),43.801727,-79.210373,2004
4,-79.254334,43.835884,2354,GO-20142417548,2014-07-02T20:52:00.000Z,2014-07-02T20:57:00.000Z,Commercial,1430,100,Assault,2014,July,2,183,Wednesday,20,2014.0,July,2.0,183.0,Wednesday,20,Assault,D42,130,Milliken (130),43.835884,-79.254334,2005


In [72]:
df_c = df_crime[["MCI", "Neighbourhood", "occurrencedayofyear"]]
crime_df = df_crime.groupby("Neighbourhood").count()


In [62]:
limit = 500
df_crime = df_crime.iloc[7:limit, :]

In [63]:
latitude = 43.6532
longitude = -79.3832

## Cluster marker crime map
Finally here is a cluster marker interactive map of crime for each neighborhood. Overall, the cluster marker provides a high-level picture of crime in each neighborhood. By zooming in or out, one can view the borough and neighborhood information.

In [73]:
# create map and display it
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# display the map of Toronto
#Toronto_map


In [74]:
# instantiate a feature group for the crime dataframe
crime = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_crime.Y, df_crime.X):
    crime.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        )
    )

# add incidents to map
#Toronto_map.add_child(crime)


In [75]:
# instantiate a feature group for the crime in the dataframe
crime = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_crime.Y, df_crime.X):
    crime.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map
latitudes = list(df_crime.Y)
longitudes = list(df_crime.X)
labels = list(df_crime.MCI)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(Toronto_map)    
    
# add crime to map
#Toronto_map.add_child(crime)

In [68]:
from folium import plugins

# let's start again with a clean copy of the map of Toronto
Toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

# instantiate a mark cluster object for the incidents in the dataframe
crime = plugins.MarkerCluster().add_to(Toronto_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df_crime.Y, df_crime.X, df_crime.MCI):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(crime)

# display map
Toronto_map


### 4.0 Conclusion
After exploring various aspects of this anaylsis, we have gained better insight than what we have started with. Based on the criteria listed as a requirement, any of the 37 neighborhood in downtown Toronto appears to be ideal for DexTR's next office location. 
It's ideally located in the city for both locals and business travelers; it's also a hub for young professionals, which is a significant percent of start-up and tech employees.
According to torontorental.com the top ten best neighborhoods were found in our borough. These neighborhoods also score above ninety percent in walk, bike, and transit score.
The inverse, downtown Toronto can be quite expensive if the start-up or the company is expense conscious.